In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import math
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Plot.BodeDiagram import bode_diagram_gain

import schemdraw
import schemdraw.elements as elm
from schemdraw import logic
from schemdraw import dsp

## setup spice library path
directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)
## set the project directory as directory_path
directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    __R = 100@u_kΩ

    def __init__(self, R=100@u_kΩ, w=0.4, name='voltage_divider'):
        self.__R__ = R
        SubCircuit.__init__(self, name, *self.__nodes__)
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )
        
    def wiper(self, w) :
        if w == 0 :
            self.R1.resistance = self.__R * 0.0000001
            self.R2.resistance = self.__R * 0.9999999
        elif w == 1 :
            self.R1.resistance = self.__R * 0.9999999
            self.R2.resistance = self.__R * 0.0000001
        else :
            self.R1.resistance = self.__R * w
            self.R2.resistance = self.__R * (1.0-w)

class DMMT3904W(SubCircuitFactory):
    __name__ = 'DMMT3904W'
    __nodes__ = ('1', '2', '3', '4', '5', '6' )
    def __init__(self, name='DMMT3904W'):
        self.__name__ = name
        SubCircuit.__init__(self, name, *self.__nodes__)
        self.BJT(101, '6', '1', '5', model="BC846B")
        self.BJT(102, '3', '2', '4', model="BC846B")

class DMMT3906W(SubCircuitFactory):
    __name__ = 'DMMT3906W'
    __nodes__ = ('1', '2', '3', '4', '5', '6' )
    def __init__(self, name='DMMT3906W'):
        self.__name__ = name
        SubCircuit.__init__(self, name, *self.__nodes__)
        self.BJT(103, '6', '1', '5', model="BC556B")
        self.BJT(104, '3', '2', '4', model="BC556B")
        

# construction

The AD/AR contains two envelope circuits and one pulse timer. Both will have a trigger at the end of the envelope or delayed pulse. The end of envelope is not connected to another input, those must be patched. The output of the delayed pulse is connected to the input of the first envelope.

In [ ]:
d = schemdraw.Drawing(unit=2, inches_per_unit=0.4, lw=0.7, fontsize=9)

d += dsp.Square().label('$\curlywedge$')

d.draw()


In [ ]:
d = schemdraw.Drawing(unit=2, inches_per_unit=0.4, lw=0.7, fontsize=9)

d += (op1 := elm.Opamp().at([5,0]).flip().label('$U_1$'))
d += elm.Line().left().at(op1.in1).length(d.unit/2).label('3', color='Grey')
d += elm.Resistor().down().length(d.unit/2).label('R5\n27k')
d += elm.Ground()

d += elm.Line().length(d.unit/2).left().at(op1.in2).label('1', color='Grey')
d += elm.Dot()
d.push()
d += elm.Line().length(d.unit/2).left()
d += ( Dot0 := elm.Dot())
d += elm.Resistor().label('R2\n27k')
d += elm.Dot()
d.push()
d += elm.Resistor().label('R1\n180k').down()
d += elm.Ground()
d.pop()
d += elm.Line().length(d.unit/2).left()
d += elm.Dot(open=True).label('trigger', loc="lft")
d.pop()
d += elm.Line().up().length(d.unit)
d += ( R3 := elm.Resistor().right().length(d.unit*1.75).label('R6\n470k'))
d += elm.Line().down().toy(op1.out)
d += elm.Dot()
d.push()
d += elm.Line().left().tox(op1.out).label('2', color='Grey')

d.pop()
d += elm.Diode().down().label('D1')
d += elm.Diode().down().label('D2').label('4', loc='start', color='Grey', ofst=(-.1, -0.2) )
d += elm.Dot()
d.push()

d += elm.Resistor().down().label('R7\n2.2k')
d += ( Dot1 := elm.Dot()).label('5', loc='left', color='Grey')
d += elm.Resistor().down().label('R8\n47k')
d += elm.Ground()

d.pop()
d += elm.Line().right().label('6', color='Grey')
d += (op2 := elm.Opamp().anchor('in2').label('$U_2$'))

d += elm.Line().at(op2.in1).left().length(d.unit/2).label('7', color='Grey')
d += elm.Line().up()
d += elm.Dot()
d.push()
d += elm.Diode().right().reverse().length(d.unit*2).label('D4')
d += elm.Dot()

d += elm.Dot()
d += elm.Diode().reverse().label('D6').label('10', loc='end', color='Grey', ofst=(0, .2) )
d += elm.ResistorVarIEEE().label('RV2\n500k')
d += elm.Dot()

d.pop()
d += elm.Line().up()
d += ( Dot2 := elm.Dot())
d += elm.Diode().right().length(d.unit*2).label('D3')
d += elm.Dot()
d.push()
d += elm.Diode().label('D5').label('9', loc='end', color='Grey', ofst=(0, .2) )
d += elm.ResistorVarIEEE().label('RV1\n100k')

d += elm.Line().down().length(d.unit*2)
d += elm.Dot()
d.push()
d += elm.Capacitor().label('C1')
d += elm.Ground()

d.pop()
d += elm.Line().right().length(d.unit).label('11', color='Grey')
d += (op3 := elm.Opamp().anchor('in2').label('$U_3$'))

d.pop()
d += elm.Line().down().toy(op2.out)
d += elm.Line().left().tox(op2.out).label('8', color='Grey')

d += elm.Line().at(op3.in1).length(d.unit/2).left()

d += elm.Line().up().toy(R3.end)
d += elm.Dot()
d.push()
d += elm.Resistor().left().label('R9\n10k').tox(Dot2.end)
d += elm.Line().down().toy(Dot2.end)

d.pop()
d += elm.Line().right().length(d.unit*2)
d += elm.Line().down().toy(op3.out)
d += elm.Dot()
d.push()

d += ( D3 := elm.Line().left().length(d.unit*0.4))
d += elm.Dot().label('12', color='Grey')
d += elm.Line().left().tox(op3.out)

d.pop()
d += elm.Resistor().down().label('R10\n2.2k', loc='bottom')
d += elm.Dot()
d.push()
d += elm.Resistor().down().label('R11\n2.2k', loc='bottom')
d += elm.Ground()

d.pop()
d += elm.Line().right().length(d.unit/2)
d += elm.Dot(open=True).label('out', loc='right')

d += elm.Line().at(Dot0.end).down().length(d.unit*4)
d += elm.Resistor().label('R3\n390k')
d += elm.Dot().label('14', loc='left', color='Grey')
d.push()
d += elm.Resistor().label('R4\n47k')
d += elm.Ground()

d.pop()
d += elm.Diode().right().length(d.unit*3).label('D7').label('13', color='Grey', ofst=(2.5, 0))
#d += elm.Line().right().length(d.unit*3).label('14', color='Grey')
d += (op4 := elm.Opamp().anchor('out').label('$U_4$')).left()

d += elm.Line().right().at(op4.in1).tox(D3.end)
d += elm.Line().up().toy(D3.end)

d += elm.Line().right().at(op4.in2)
d += elm.Line().up().toy(Dot1.end)
d += elm.Line().left().tox(Dot1.end)

d.draw()




# references

https://bitbucket.org/folle/arduino-fm-synthesizer/src/master/Arduino_Due_FM_Synthesizer/Arduino_Due_FM_Synthesizer.ino

https://modwiggler.com/forum/viewtopic.php?t=134469